Summary: Script producing plots and descriptive statistics for `multi_level_illness_model_write_up.Rmd`.

Notes:
*same essential issues as described in `multi_level_illness_models_experiments`*
* All models incorrectly assume independence of within-school observations and thus variance estimates/confidence intervals are not accurate. The observations are weekly rates, thus are auto-correlated. This relationship is explored in the notebook `autocorr_tests.ipynb` and in the latter few plots of this notebook.
* Bayesian sampling is used to estimate the model parameters (this method is later abandoned as being unnecessarily complex). 
* Multiple specifications of the logistic regression model are tested and the lowest BIC selected. This is somewhat arbitrary model selection, and several of the model variants are obviously functionally flawed i.e. increased CO2 decreasing illnesses or greater covid rates in the population decreasing illnesses etc.
* LOESS curves are again used as smoothed mean estimates. The fit is arbitrary - it's based on the default settings of ggplot, and are not based on an interpretable or statistically sound model, thus shouldn't be considered reliable when interpreting the effects of either ACTs (or absence of ACTs). 
* lm smoothed mean estimates used to plot lines of best fit - selection of confidence intervals again arbitrary based on default settings of ggplot
* None of the analyses explore the high-leverage outlying observations - can be seen in residuals plots 

In [ ]:
library(tidyverse)
library(tidybayes)
library(bigrquery)
library(brms)
library(ggmcmc)
library(loo)
library(lubridate)
library(gridExtra)
library(grid)
library(bayesplot)
library(marginaleffects)
library(table1)

bq_auth()

hepa_school_codes <- c("H01", "H02", "H03", "H04", "H05", "H06", "H07", "H08", 
                       "H09", "H10", "H11")
# removed C01 due to primary/secondary attendance issue
control_school_codes <- c("C01", "C02", "C03", "C04", "C05", "C09", "C10", 
                          "C11", "C12", "C13", "C14")
study_schools <- c(hepa_school_codes, control_school_codes)

project_id="yhcr-prd-phm-bia-core" 
attendance_sql <- "SELECT * FROM `yhcr-prd-phm-bia-core.CY_CLASS_ACT.attendance`"
attendance_table <- bq_project_query(project_id, attendance_sql)

attendance <- bq_table_download(attendance_table) %>%
    filter(School_AnonID %in% study_schools) %>% 
    filter(WeekStart <= as.Date("2022-04-01")) %>%
    filter(pct_in_school > 0) 

attendance_data_threshold <- attendance$Unk / (attendance$pupils *  14) >= 0.01
attendance[attendance_data_threshold, "prop_absent_ill"] <- NA

attendance$arm <- "None"
attendance[attendance$School_AnonID %in% control_school_codes, "arm"] <- "Control" 
attendance[attendance$School_AnonID %in% hepa_school_codes, "arm"] <- "HEPA"

msoa_rates_link <- "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E08000032&metric=newCasesBySpecimenDateRollingRate&format=csv"
keep_cols <- c("msoa", "WeekStart", "covid_msoa_rate")
msoa_rates <- read_csv(msoa_rates_link) %>%
    mutate(WeekStart = date - 5) %>%
    rename(covid_msoa_rate = newCasesBySpecimenDateRollingRate, msoa = areaCode) %>%
    select(all_of(keep_cols))

cov_age_rate_link <- "https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=E08000032&metric=newCasesBySpecimenDateAgeDemographics&format=csv&release=2022-04-29"
cov_age_rates <- read_csv(cov_age_rate_link) %>%
    filter(age %in% c("05_09", "10_14")) %>%
    filter(weekdays(date) == "Saturday") %>%
    mutate(WeekStart = date - 5) %>%
    group_by(WeekStart) %>%
    summarise(covid_age_rate = mean(rollingRate))

attendance <- attendance %>% 
    left_join(msoa_rates, by=c("WeekStart", "msoa")) %>%
    left_join(cov_age_rates, by=c("WeekStart")) 

co2_sql <- "SELECT School_ID, week_start, co2_mean FROM `yhcr-prd-phm-bia-core.CY_CLASS_ACT.stats_week_school`"
co2_table <- bq_project_query(project_id, co2_sql)
co2 <- bq_table_download(co2_table)

co2 <- co2 %>% 
    rename(WeekStart = week_start, School_AnonID = School_ID) 

attendance <- attendance %>% 
    left_join(co2, by=c("WeekStart", "School_AnonID"))

thresholds <- attendance %>% group_by(WeekStart) %>% 
    summarise(outlier_threshold = quantile(prop_absent_ill, p=0.75, na.rm=TRUE) + 3*(IQR(prop_absent_ill, na.rm=TRUE)))

attendance <- attendance %>% 
    left_join(thresholds, by="WeekStart") %>% 
    mutate(covid_age_rate_scaled = covid_age_rate / max(covid_age_rate), 
           covid_msoa_rate_scaled = covid_msoa_rate / max(covid_msoa_rate, na.rm=TRUE), 
           prop_absent_ill_scaled = prop_absent_ill / max(prop_absent_ill, na.rm=TRUE), 
           co2_mean_scaled = co2_mean / max(co2_mean, na.rm=TRUE), 
           is_outlier = prop_absent_ill > outlier_threshold)

In [ ]:
max(attendance$WeekStart)

In [ ]:
table_1 <- table1(~ prop_absent_ill +  co2_mean + covid_msoa_rate + 
                  covid_age_rate | arm, data=attendance)
as.data.frame(table_1)

In [ ]:
return_illness_model <- function(data, covariates) {
    model_string <- append("prop_absent_ill ~ 1 + arm", covariates)
    model_string <- append(model_string, "(1 | School_AnonID)")
    model_string <- paste(model_string, collapse=" + ")
    model <- brm(model_string, data = data, family=Gamma(link="log"),  
                 warmup = 500, iter = 2000, chains = 5, init = 1, cores = 5)
    model <- add_criterion(model, "loo")
    return(model)
}

msoa_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_msoa_rate_scaled"
)
int_msoa_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_msoa_rate_scaled*arm"
)
co2_model <- return_illness_model(
    attendance %>% na.omit(),  
    "co2_mean_scaled"
)
int_co2_model <- return_illness_model(
    attendance %>% na.omit(),  
    "co2_mean_scaled*arm"
)
msoa_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled", "co2_mean_scaled")
)
int_msoa_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled*arm", "co2_mean_scaled")
)
msoa_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled", "co2_mean_scaled*arm")
)
int_msoa_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled*arm", "co2_mean_scaled*arm")
)
age_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_age_rate_scaled"
)
int_age_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_age_rate_scaled*arm"
)
age_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled", "co2_mean_scaled")
)
int_age_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled*arm", "co2_mean_scaled")
)
age_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled", "co2_mean_scaled*arm")
)
int_age_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled*arm", "co2_mean_scaled*arm")
)

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 5, repr.plot.res = 200)
plot_top <- ggplot(attendance) +  
    geom_boxplot(aes(x=WeekStart,  
                     y=covid_msoa_rate,  
                     group=WeekStart,
                     color="MSOA Rate")) + 
    labs(x="", y="+ve Test Rate")  + 
    theme_classic() +
    scale_colour_manual("", 
                      breaks = c("MSOA Rate", "4-14 y/o rate"),
                      values = c("#247afd", "#cf0234")) +
    geom_line(aes(x=WeekStart, y=covid_age_rate, color="4-14 y/o rate"))

plot_bottom <- attendance %>%  
    filter(!is_outlier) %>% 
    ggplot() + 
    geom_boxplot(aes(x=WeekStart,    
                     y=prop_absent_ill,    
                     group=interaction(WeekStart))) + 
    labs(x="Week Start", y="Illness Ratio")  + 
    theme_classic() +
    theme(plot.margin = unit(c(0,3.2,0,0.3), "cm")) 

rates_plot<- grid.arrange(plot_top, plot_bottom, 
                          top=textGrob("Trend of +ve Test Rates and Illness Ratios Over Study Period", 
                                       gp=gpar(fontsize=15)),
                          nrow=2, ncol=1, heights=c(1,1))
ggsave("plots/rates.jpg", rates_plot, width=7.5, height=5)

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 7.5, repr.plot.res = 200)

msoa_l_plot <- ggplot(attendance, aes(x=log(covid_msoa_rate),  
                                 y=log(prop_absent_ill))) +  
    geom_jitter(alpha=0.4, width=0.03) +
    theme_classic() +
    labs(x="log MSOA rate", y="log Illness Ratio") +
    geom_smooth(method="lm")

msoa_r_plot <- ggplot(attendance, aes(x=log(covid_msoa_rate),  
                                 y=log(prop_absent_ill),  
                                 color=arm)) +  
    geom_jitter(alpha=0.4, width=0.03) +
    theme_classic() +
    labs(x="log MSOA rate", y="log Illness Ratio")  + 
    geom_smooth(method="lm")

msoa_rate_v_illness <- grid.arrange(
    msoa_l_plot, msoa_r_plot,   
    top=textGrob("MSOA-level - All Ages:",    
                 gp=gpar(fontsize=11),
                 x=0.16), 
    ncol=2, widths=c(0.4,0.6))

age_l_plot <- ggplot(attendance, aes(x=log(covid_age_rate),  
                                 y=log(prop_absent_ill))) +  
    geom_jitter(alpha=0.4, width=0.03) +
    theme_classic() +
    labs(x="log 4-14 y/o rate", y="log Illness Ratio")  + 
    geom_smooth(method="lm")

age_r_plot <- ggplot(attendance, aes(x=log(covid_age_rate),  
                                 y=log(prop_absent_ill),  
                                 color=arm)) +  
    geom_jitter(alpha=0.4, width=0.03) +
    theme_classic() +
    labs(x="log 4-14 y/o rate", y="log Illness Ratio")  + 
    geom_smooth(method="lm");

age_rate_v_illness <- grid.arrange(
    age_l_plot, age_r_plot,   
    top=textGrob("Bradford-level - 4-14 Year Olds:",   
                 gp=gpar(fontsize=11),
                 x=0.2), 
    ncol=2, widths=c(0.4,0.6))

rates_v_illness <- grid.arrange(
    msoa_l_plot, msoa_r_plot, age_l_plot, age_r_plot,  
    top=textGrob("Correlation of +ve Covid Test Metrics and Illness Absence Ratios",    
                 gp=gpar(fontsize=13)), 
    nrow=2, ncol=2, widths=c(0.4, 0.6))
ggsave("plots/rates_v_illness.jpg", rates_v_illness, width=7.5, height=7.5)

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 5, repr.plot.res = 200)
co2_means <- ggplot(attendance) +  
    geom_boxplot(aes(x=WeekStart, 
                     y=co2_mean, 
                     group=WeekStart)) + 
    theme_classic() +
    labs(x="Week Start", y="Mean CO²",
         title="Trend of Mean CO² Concentrations Over Study Period")  + 
    theme(plot.title = element_text(hjust = 0.5)) +
    geom_smooth(aes(x=WeekStart, y=co2_mean))

ggsave("plots/co2_means.jpg", co2_means, width=7.5, height=5)
co2_means

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 3, repr.plot.res = 200)
l_plot <- ggplot(attendance, aes(x=co2_mean,  
                                 y=log(prop_absent_ill))) +  
    geom_jitter(alpha=0.4, width=0.03) +
    theme_classic() +
    labs(x="Mean CO²", y="log Illness Ratio")  + 
    geom_smooth(method="lm")

r_plot <- ggplot(attendance, aes(x=co2_mean,  
                                 y=log(prop_absent_ill),  
                                 color=arm)) +  
    geom_jitter(alpha=0.4, width=0.03) +
    theme_classic() +
    labs(x="Mean CO²", y="log Illness Ratio")  + 
    geom_smooth(method="lm")
co2_v_illness <- grid.arrange(l_plot, r_plot,  
                              top=textGrob("Correlation of Mean CO² Concentration and Illness Absence Ratios",   
                                           gp=gpar(fontsize=12)),
                              ncol=2, widths=c(0.4,0.6));
ggsave("plots/co2_v_illness.jpg", co2_v_illness, width=7.5, height=3)

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 6, repr.plot.res = 200)

plot_top <- ggplot(attendance) + 
            geom_smooth(mapping=aes(x=WeekStart, y=prop_absent_ill, color=arm), 
                        method="loess")  + 
            labs(x="", y="Illness Ratio") + 
            ggtitle("LOESS Curves of Illness Ratio by Date: HEPA vs Control") + 
            theme_classic() +
            theme(plot.title = element_text(hjust = 0.5))

plot_bottom <- ggplot(
    data=attendance %>%   
    mutate(month_year = as.Date(format(attendance$WeekStart, "%Y-%m-01")))) + 
               geom_boxplot(mapping=aes(x=month_year, 
                                        y=prop_absent_ill, 
                                        color=arm,   
                                        group=interaction(month_year, arm))) + 
               labs(x="", y="Illness Ratio") + 
               ggtitle("Distribution of Illness Ratios by Month and Study Arm")  + 
               theme_classic() +
               theme(plot.title = element_text(hjust = 0.5))

hepa_v_illness <- grid.arrange(plot_top, plot_bottom, nrow=2, heights=c(1.6,2))
ggsave("plots/hepa_v_illness.jpg", hepa_v_illness, width=7.5, height=6)

In [ ]:
loos <- loo_compare(msoa_model,   
                    int_msoa_model,   
                    co2_model,   
                    int_co2_model,   
                    msoa_co2_model,   
                    int_msoa_co2_model,   
                    msoa_int_co2_model,   
                    int_msoa_int_co2_model,   
                    age_model,   
                    int_age_model,   
                    age_co2_model,   
                    int_age_co2_model,   
                    age_int_co2_model,   
                    int_age_int_co2_model, 
                    criterion="loo")
loos <- round(loos, digits=2)
loos

In [ ]:
write.csv(loos, "loos.csv")

In [ ]:
msoa_int_co2_model

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 8, repr.plot.res = 200)
plot_posterior_densities <- function(model) {
    ggs(model) %>%   
        filter(!grepl("\\[", Parameter)) %>%  
        filter(grepl("b_", Parameter)) %>%
    ggplot(aes(x=value, y=Parameter, slab_color=Parameter)) +
    stat_dotsinterval(show.legend=FALSE, quantiles=300) +
    geom_vline(xintercept=0, linetype="dashed", col="red") + 
    labs(x="", y="Parameter") + 
    theme_classic() 
}

pos_density_plot <- plot_posterior_densities(msoa_int_co2_model)
ggsave("plots/pos_density_plot.jpg", pos_density_plot, width=6, height=8)
pos_density_plot

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 4, repr.plot.res = 200)
cbbPalette <- c("#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7")
co2_min <- min(attendance$co2_mean_scaled, na.rm=TRUE)
preds <- predictions(
    msoa_int_co2_model,  
    newdata=datagrid(arm=c("Control", "HEPA"),  
                     co2_mean_scaled=seq(co2_min,1,0.05),  
                     covid_msoa_rate_scaled=seq(co2_min,1,0.05))) %>%
    mutate(co2_mean = co2_mean_scaled * max(attendance$co2_mean, na.rm=TRUE)) 
co2_preds <- ggplot(preds, 
                    aes(y=predicted,  
                        x=co2_mean,  
                        color=arm)) + 
             geom_smooth(alpha=0.5, method="lm") +
             labs(x="CO² Mean", y="Predicted Illness Ratio") + 
             theme_classic() +
             theme(plot.title = element_text(hjust = 0.5),
                   axis.ticks.length = unit(0.15, "cm")) +
             scale_color_manual(values=c("#E69F00", "#56B4E9"))
ggsave("plots/co2_preds.jpg", co2_preds, width=7.5, height=4)
co2_preds

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 4, repr.plot.res = 200)
cbbPalette <- c("#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7")
school_ids = attendance %>% select(School_AnonID) %>% unique() %>% unlist() %>% unname()
co2_min <- min(attendance$co2_mean_scaled, na.rm=TRUE)
rate_min <- min(attendance$covid_msoa_rate_scaled, na.rm=TRUE)
n_samples <- 4000
new_data <- data.frame(
    co2_mean_scaled = rep(runif(n_samples, co2_min, 1), 2),  
    covid_msoa_rate_scaled = rep(runif(n_samples, rate_min, 1), 2), 
    School_AnonID = rep(sample(school_ids, n_samples, replace=TRUE), 2), 
    arm = c(rep("Control", n_samples), rep("HEPA", n_samples))
) 
preds <- predictions(msoa_int_co2_model, newdata=new_data) %>%
    mutate(co2_mean = co2_mean_scaled * max(attendance$co2_mean, na.rm=TRUE)) %>%
    select(arm, co2_mean, covid_msoa_rate_scaled, School_AnonID, predicted) %>% 
    spread(arm, predicted) %>%
    mutate(diff = Control - HEPA)
co2_preds <- ggplot(preds, 
                    aes(y=diff,  
                        x=co2_mean)) + 
             geom_point(alpha=0.3, color="orange") +
             geom_smooth(method="lm", se=FALSE) +
             labs(x="Weekly Average CO₂ (ppm)", y="Difference: Predicted Illness Ratio")+ 
             theme_classic() +
             theme(plot.title = element_text(hjust = 0.5),
                   axis.ticks.length.x = unit(0.25, "cm")) +
             scale_color_manual(values=c("#E69F00", "#56B4E9")) +
             scale_x_continuous(breaks = seq(800, 2000, 200))
ggsave("plots/co2_preds.jpg", co2_preds, width=7.5, height=4)
co2_preds

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 3, repr.plot.res = 200)
pred <- predictions(msoa_int_co2_model,
                    re_formula = NA,
                    newdata = datagrid(arm = c("Control", "HEPA"))) |>
        posteriordraws()

mfx <- marginaleffects(msoa_int_co2_model,
                       re_formula = NA,
                       variables = "arm") |>
       posteriordraws()

In [ ]:
plot_l <- ggplot(mfx, aes(x = draw)) +  
              stat_dotsinterval(point_interval="mean_hdi", slab_alpha = .5, quantiles=300)  + 
              geom_vline(xintercept=0, linetype="dashed", col="red") +  
              labs(x = "Predicted Illness Ratio Difference", 
                   y = "Prediction Density", 
                   fill = "Study Arm") +
              theme(legend.position = "none") +
              theme_classic() 

plot_r <- ggplot(pred, aes(x = draw, fill = arm)) +
         stat_dotsinterval(point_interval="mean_hdi", slab_alpha = .5, quantiles=300) +
         labs(x = "Predicted Illness Ratio",
              y = "Posterior density",
              fill = "Study Arm") + 
         theme_classic() +
         scale_fill_manual(values=c("#E69F00", "#56B4E9"))

margin_effects <- grid.arrange(
    plot_l, plot_r, 
    ncol=2, widths=c(0.4, 0.6))
ggsave("plots/margin_effects.jpg", margin_effects, width=7.5, height=4)

In [ ]:
margin_effects

In [ ]:
mfx %>% pull(draw) |> hdi()

In [ ]:
mean(mfx %>% pull(draw))

In [ ]:
pred %>% filter(arm == "Control") %>% pull(draw) |> hdi()

In [ ]:
mean(pred %>% filter(arm == "Control") %>% pull(draw))

In [ ]:
pred %>% filter(arm == "HEPA") %>% pull(draw) |> hdi()

In [ ]:
mean(pred %>% filter(arm == "HEPA") %>% pull(draw))

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
attendance %>%
  add_residual_draws(msoa_int_co2_model) %>%
  ggplot(aes(x = .row, y = .residual)) +
  stat_pointinterval()

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
attendance %>%
  add_residual_draws(msoa_int_co2_model) %>%
  median_qi() %>%
  ggplot(aes(sample = .residual)) +
  geom_qq() +
  geom_qq_line()

In [ ]:
attendance %>% na.omit() %>%
  add_residual_draws(msoa_int_co2_model) %>%
    slice_sample(n=5) %>%
    ggplot(aes(x=WeekStart, y=.residual)) +
    geom_point() +
    geom_smooth()

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 3, repr.plot.res = 200)
attendance <- attendance %>% 
    mutate(day = yday(WeekStart)) %>% 
    mutate(winter_period = cos(day/365*2*pi),
           autumn_period = cos((day+121)/365*2*pi),
           summer_period = cos((day+182)/365*2*pi),
           spring_period = cos((day-121)/365*2*pi))

attendance %>% 
    ggplot() +
    geom_smooth(aes(x=WeekStart, y=winter_period), color="blue") +
    geom_smooth(aes(x=WeekStart, y=autumn_period), color="orange") +
    geom_smooth(aes(x=WeekStart, y=spring_period), color="yellow") 

In [ ]:
test_model_formula <- bf(prop_absent_ill ~ 1 + 
                         arm +
                         (1 | School_AnonID))
test_model <- brm(test_model_formula, 
                  data = attendance %>% na.omit(), 
                  family=Gamma(link="log"),   
                  warmup = 500, iter = 2000, chains = 5, init = 1, cores = 5)
test_model <- add_criterion(test_model, "loo")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
attendance %>%
  add_residual_draws(test_model) %>%
  ggplot(aes(x = .row, y = .residual)) +
  stat_pointinterval()

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
attendance %>%
  add_residual_draws(test_model) %>%
  median_qi() %>%
  ggplot(aes(sample = .residual)) +
  geom_qq() +
  geom_qq_line()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 300)
attendance %>% na.omit() %>%
  add_residual_draws(test_model) %>%
    slice_sample(n=5) %>%
    ggplot(aes(x=WeekStart, y=.residual)) +
    geom_point() +
    geom_jitter()

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 8, repr.plot.res = 200)
plot_posterior_densities <- function(model) {
    ggs(model) %>%   
        filter(!grepl("\\[", Parameter)) %>%  
        filter(grepl("b_", Parameter)) %>%
    ggplot(aes(x=value, y=Parameter, slab_color=Parameter)) +
    stat_dotsinterval(show.legend=FALSE, quantiles=300) +
    geom_vline(xintercept=0, linetype="dashed", col="red") + 
    labs(x="", y="Parameter") + 
    theme_classic() 
}

pos_density_plot <- plot_posterior_densities(test_model)
ggsave("plots/pos_density_plot.jpg", pos_density_plot, width=6, height=8)
pos_density_plot

# ARMA errors
```
test_model_formula <- bf(prop_absent_ill ~ 1 + 
                         arm*co2_mean_scaled + covid_msoa_rate_scaled + 
                         (1 | School_AnonID), autocor= ~ arma(gr=School_AnonID,time=WeekStart))
test_model <- brm(test_model_formula, 
                  data = attendance %>% na.omit(), 
                  family=Gamma(link="log"),   
                  warmup = 500, iter = 2000, chains = 5, init = 1, cores = 5)
test_model <- add_criterion(test_model, "loo")
```